# Second-Order Methods in TensorFlow - Part 1

Doing second-order methods in tensorflow is not well-supported, so it's easy to make a mistake.

To get a better handle on using second-order methods for neural networks, where the ground truth is unclear or hard to calculate, I'm working through second-order methods for a case where we have ground truth: quadratic forms.

A quadratic form is a polynomial of degree two over an $n$-dimensional input. They are calculated as

$$
\mathbf{x}^\intercal Q \mathbf{x}
$$

Below, I walk through the gradient and Hessian calculations.

In [35]:
import tensorflow as tf

import matplotlib.pyplot as plt

import numpy as np

from collections import namedtuple

In [36]:
QuadraticFormGraph = namedtuple("QuadraticFormGraph", ["graph", "graph_dictionary"])

In [155]:
def make_quadratic_form_graph(matrix):
    assert matrix.shape[0] == matrix.shape[1], "only square matrices can be quadratic forms"
    dimension = matrix.shape[0]
    
    graph = tf.Graph()
    
    with graph.as_default():
        quadratic_form = tf.constant(matrix, name='quadratic_form')
    
        input_placeholder = tf.placeholder(tf.float64, shape=[dimension], name='input_elements')
        
        input_vector = tf.expand_dims(input_placeholder, axis=1)
    
        output_op = tf.squeeze(tf.matmul(input_vector,
                                  tf.matmul(quadratic_form, input_vector),
                                         transpose_a=True,
                                  name='output'),
                        name='squeezed_output')
        
        gradient_ops = tf.gradients(output_op, input_placeholder)
        
        hessian_op = tf.hessians(output_op, input_placeholder)
        
        graph_dictionary = {"inputs": input_placeholder,
                           "output": output_op,
                           "gradients": gradient_ops,
                           "hessian": hessian_op,
                           #"alt_hessian": alt_hessian_ops
                           }
    
    return QuadraticFormGraph(graph, graph_dictionary)

In [156]:
def get_result(result_key, input_vector, quadratic_form_graph):
    
    graph, graph_dictionary = quadratic_form_graph
    result_op = graph_dictionary[result_key]
    input_placeholders = graph_dictionary["inputs"]
    
    input_feed_dict = make_feed_dict(input_placeholders, input_vector)
    
    result = run(graph, result_op, input_feed_dict)
    
    return result

def run(graph, op, input_feed_dict):
    
    with graph.as_default():
        with tf.Session() as sess:
            result = sess.run(op, feed_dict = input_feed_dict)
            
    return result

def make_feed_dict(input_placeholders, input_vector):
    
    feed_dict = {input_placeholders:input_vector}
    
    return feed_dict

## Testing with Identity Matrix

The simplest case, useful for working out basic bugs and flaws in reasoning, is the identity matrix, because  the quadratic form it defines is the squared $\ell_2$ norm:

$$
\mathbf{x}^\intercal I \mathbf{x} = \mathbf{x}^\intercal \mathbf{x} = \sum_i x_i \cdot x_i
$$

In [157]:
N = 2

identity_matrix = np.eye(N)

identity_quadratic_form_graph = make_quadratic_form_graph(identity_matrix)

input_vector = np.sqrt([1/2,1/2])

In [158]:
out = get_result("output", input_vector, identity_quadratic_form_graph)

assert np.isclose(np.sum(np.square(input_vector)), out)

The gradient is just twice the input:

$$
\nabla_\mathbf{x} \mathbf{x}^\intercal I \mathbf{x} = \nabla_x \sum_i x_i\cdot x_i = 2\cdot I\cdot \mathbf{x}
$$

In [159]:
gradient = np.squeeze(get_result("gradients", input_vector, identity_quadratic_form_graph))

gradient

array([ 1.41421356,  1.41421356])

In [160]:
assert np.allclose(input_vector.T*2, gradient)

The Hessian matrix is just $2\cdot I$.

In [161]:
get_result("hessian", input_vector, identity_quadratic_form_graph)

[array([[ 2.,  0.],
        [ 0.,  2.]])]

In [162]:
assert np.allclose(hessian, 2*np.eye(N))

## Random Matrix

Of course, to really ensure that our code is correct, we should test on less-symmetric problems, e.g. matrices from the Gaussian ensemble.

Generically, the gradient of $\mathbf{x}^\intercal Q \mathbf{x}$ is

$$
\nabla_\mathbf{x} \mathbf{x}^\intercal Q \mathbf{x} = (Q + Q^\intercal)\mathbf{x}
$$

Which we can get from the definition of the derivative, following
[this derivation from StackOverflow](https://math.stackexchange.com/questions/239207/hessian-matrix-of-a-quadratic-form).

Writing $f(\mathbf{x})$ for $\mathbf{x}^\intercal Q \mathbf{x}$, our goal is to find $\nabla_\mathbf{x}f(\mathbf{x})$ such that

$$\begin{align}
f(\mathbf{x}+\epsilon) &= f(\mathbf{x}) + \nabla_\mathbf{x}f(\mathbf{x})\epsilon + o(\epsilon) \\
\end{align}$$

as the norm of $\epsilon$ goes to $0$.
We expand the left-hand side:

$$\begin{align}
f(\mathbf{x}+\epsilon) &= (\mathbf{x}+\epsilon)^\intercal Q (\mathbf{x}+\epsilon) \\
&= \mathbf{x}^\intercal Q \mathbf{x}
+ \mathbf{x}^\intercal Q \epsilon + \epsilon^\intercal Q \mathbf{x}
+ \epsilon^\intercal Q \epsilon
\end{align}$$

Using the rules for transposition and distribution, we can rewrite this as

$$\begin{align}
f(\mathbf{x}+\epsilon)
&= \mathbf{x}^\intercal Q \mathbf{x}
+ \mathbf{x}^\intercal Q \epsilon + \epsilon^\intercal Q \mathbf{x}
+ \epsilon^\intercal Q \epsilon \\
&= \mathbf{x}^\intercal Q \mathbf{x}
+ \mathbf{x}^\intercal Q \epsilon + (\epsilon^\intercal Q \mathbf{x})^\intercal
+ \epsilon^\intercal Q \epsilon \\
&= \mathbf{x}^\intercal Q \mathbf{x}
+ \mathbf{x}^\intercal Q \epsilon + \mathbf{x}^\intercal Q^\intercal \epsilon
+ \epsilon^\intercal Q \epsilon \\
&= \mathbf{x}^\intercal Q \mathbf{x}
+ \mathbf{x}^\intercal (Q+Q^\intercal) \epsilon
+ \epsilon^\intercal Q \epsilon \\
\end{align}$$

Comparing this to the definition of the derivative above,
we see that we have a term $f(\mathbf{x})$ and a term dominated by $\epsilon$,
leaving the middle term, sans $\epsilon$, to be our derivative:

$$
\nabla_\mathbf{x} \mathbf{x}^\intercal Q \mathbf{x} = \mathbf{x}^\intercal (Q + Q^\intercal)
$$

Obviously, the Hessian is thus $Q+Q^\intercal$.

In [163]:
N = 3

random_matrix = np.random.standard_normal(size=(N,N))

random_quadratic_form_graph = make_quadratic_form_graph(random_matrix)

input_vector = np.random.standard_normal(size=(N))

In [164]:
out = get_result("output", input_vector, random_quadratic_form_graph)
out

-0.91437031120660439

In [165]:
assert np.isclose(out, input_vector.T @ random_matrix @ input_vector)

In [166]:
gradient = np.squeeze(get_result("gradients", input_vector, random_quadratic_form_graph))
gradient

array([ 2.8263463 ,  0.50771755,  0.84111005])

In [167]:
assert np.allclose(gradient, np.squeeze((random_matrix+random_matrix.T) @ input_vector))

Again, we find that `tf.hessians` gets only the diagonal elements.

In [168]:
tf_hessian = get_result("hessian", input_vector, random_quadratic_form_graph)
tf_hessian

[array([[-4.19260739, -0.53689794, -1.73109465],
        [-0.53689794, -0.23361096, -0.43265071],
        [-1.73109465, -0.43265071,  0.29310839]])]

In [169]:
true_hessian = random_matrix+random_matrix.T
true_hessian

array([[-4.19260739, -0.53689794, -1.73109465],
       [-0.53689794, -0.23361096, -0.43265071],
       [-1.73109465, -0.43265071,  0.29310839]])

In [170]:
assert np.allclose(true_hessian, tf_hessian)